In [1]:
import gadgets
import datasets
import transformers
import pandas as pd
import os

[2023-08-13 15:59:13,005] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
ds = datasets.load_dataset("gsm8k", "main")

In [3]:
os.makedirs("../data/gsm8k", exist_ok=True)
df = {}
for split in ["train", "test"]:
    df[split] = ds[split].to_pandas()
    df[split]["chain"] = df[split].apply(gadgets.gsm8k.parse, axis=1).apply(lambda x: gadgets.markup.to_model_markup(example=x)).apply(str)
    df[split].to_json(f"../data/gsm8k/{split}.jsonl", orient="records", lines=True, force_ascii=False)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")


In [4]:
with open("../data/gsm8k/train.jsonl") as f:
    content = f.read()
with open("../data/gsm8k/test.jsonl") as f:
    content += f.read()
content.count('\u2028')

0

In [5]:
foo = pd.Series([x for x in content if not x.isascii()]).value_counts().reset_index()

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained("t5-base")
# TODO many random unicode characters that might not be in the tokenizer's vocab

/var/tmp/xkadlci2/.conda/envs/gadgets/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
foo["tokenized"] = foo["index"].apply(tokenizer.encode).apply(tokenizer.decode, skip_special_tokens=True)

In [8]:
foo

,index,count,tokenized
0,€,100,€
1,£,25,£
2,¢,25,
3,é,22,é
4,ñ,4,
5,è,4,è
6,ç,3,ç
